In [1]:
%load_ext autoreload
%autoreload 2
import qcodes as qc

matplotlib plotting not supported, try "from qcodes.plots.qcmatplotlib import MatPlot" to see the full error
pyqtgraph plotting not supported, try "from qcodes.plots.pyqtgraph import QtPlot" to see the full error


<IPython.core.display.Javascript object>

# QCoDeS config

Qcodes loads defaults and configurations at module import

In [2]:
qc.config.current_config

{'core': {'legacy_mp': False, 'loglevel': 'DEBUG'},
 'gui': {'notebook': True, 'plotlib': 'matplotlib'},
 'user': {}}

In [3]:
qc.config.defaults

{'core': {'legacy_mp': False, 'loglevel': 'DEBUG'},
 'gui': {'notebook': True, 'plotlib': 'matplotlib'},
 'user': {}}

One can inspect what the configuration options mean at runtime

In [4]:
print(qc.config.describe('core'))

controls core settings of qcodes.
Current value: {'legacy_mp': False, 'loglevel': 'DEBUG'}. Type: object. Default: Not defined.


## configuring qcodes

Defaults are the settings that are shipped with the package, which you can overwrite programmatically.

A way to customize qcodes is to write your own json files, they are expected to be  in the  directories printed below.
One will be empty because one needs to define first the environment variable in the OS.

They are ordered by "weight", meaning that the last file always wins if it's overwriting any preconfigured defaults or values in the other files.

Simply copy the file to the directories and you are good to go.

In [5]:
print("\n".join([qc.config.home_file_name, qc.config.env_file_name, qc.config.cwd_file_name]))

/home/unga/qcodesrc.json

/home/unga/src/Qcodes/docs/examples/qcodesrc.json


But the easiest way to add something to the config is to use the provided helper:

In [6]:
qc.config.add("base_location", "/dev/random", value_type="string", description="Location of data", default="/dev/random")

This will add a base_location with value /dev/random to the current config, and validate it's value to be of type string, will also set the description and what one would want to have as default.
The new entry is saved in the 'user' part of the config.

In [7]:
print(qc.config.describe('user.base_location'))

Location of data.
Current value: /dev/random. Type: string. Default: /dev/random.


## saving changes

All the changes made to the defaults are stored, and one can then decide to save them to the expected place.

In [8]:
help(qc.config.save_to_cwd)

Help on method save_to_cwd in module qcodes.config.config:

save_to_cwd() method of qcodes.config.config.Config instance
    Save files to current working dir



In [9]:
help(qc.config.save_to_env)

Help on method save_to_env in module qcodes.config.config:

save_to_env() method of qcodes.config.config.Config instance
    Save  files to env path



In [10]:
help(qc.config.save_to_home)

Help on method save_to_home in module qcodes.config.config:

save_to_home() method of qcodes.config.config.Config instance
    Save  files to home dir



### Using  a custom configured variable  in your experiment:

Simply get the value you have set before with dot notation.
For example:

In [11]:
loc_provider = qc.data.location.FormatLocation(fmt=qc.config.user.base_location)
qc.data.data_set.DataSet.location_provider=loc_provider  

##  Changing core

one can change the core values at runtime, but  there is no guarantee that they are valided.
Changing core values will also shadow the config from being updated if the defaults are updated upsteam (i.e. in the repo). So use it with a care.

In [12]:
qc.config.current_config.core.loglevel = 'INFO'

but one can maunually validate 

In [13]:
qc.config.validate()

which will raise an exception in case of bad inputs

In [14]:
qc.config.current_config.core.loglevel = 'YOLO'
qc.config.validate() 
# NOTE that you how have a broken config! 

ValidationError: 'YOLO' is not one of ['CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG']

Failed validating 'enum' in schema['properties']['core']['properties']['loglevel']:
    {'default': 'DEBUG',
     'description': 'control logging  level',
     'enum': ['CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG'],
     'type': 'string'}

On instance['core']['loglevel']:
    'YOLO'